In [259]:
import numpy as np
import pandas as pd

In [260]:
class binary():
    def __init__(self,capas=1,nodos=1):
        self.capas = capas
        self.nodos = nodos
        self.forma = (self.capas,self.nodos)
    def inicializar(self):
        self.red = []
        for capa in range(self.capas):
            w = np.random.normal(size = (self.nodos,self.nodos))
            b = np.random.normal(size = self.nodos)
            self.red.append([w,b])
        w_f = np.random.normal(size = self.nodos)
        b_f = np.random.normal(size = 1)
        self.red.append([w_f,b_f])
    def mutar(self,alfa=0.1):
        red = []
        for r in self.red:
            r0 = r[0]+alfa*np.random.normal(size = r[0].shape)
            r1 = r[1]+alfa*np.random.normal(size = r[1].shape)
            red.append([r0,r1])
        hijo = binary(self.capas,self.nodos)
        hijo.inicializar()
        hijo.red = red
        return hijo
    def cruzar(self,other):
        if self.forma == other.forma:
            hijo = binary(self.capas,self.nodos)
            hijo.inicializar()
            red = []
            for i in range(len(self.red)):
                r0 = (self.red[i][0] + other.red[i][0])/2.0
                r1 = (self.red[i][1] + other.red[i][1])/2.0
                red.append([r0,r1])
            hijo.red = red
            return hijo
        else:
            return None
    def clonar(self):
        hijo = binary(self.capas,self.nodos)
        hijo.inicializar()
        red = []
        for i in range(len(self.red)):
            r0 = self.red[i][0]
            r1 = self.red[i][1]
            red.append([r0,r1])
        hijo.red = red
        return hijo
    def sumar(self,other):
        for i in range(len(self.red)):
            self.red[i][0] += other.red[i][0]
            self.red[i][1] += other.red[i][1]
    def normalizar(self,n_pasos):
        for i in range(len(self.red)):
            self.red[i][0] /= n_pasos
            self.red[i][1] /= n_pasos
    def distancia(self,other):
        if self.forma == other.forma:
            distancia = 0
            for i in range(len(self.red)):
                r0 = (self.red[i][0] - other.red[i][0])**2
                r1 = (self.red[i][1] - other.red[i][1])**2
                distancia = distancia + r0.sum().sum()+r1.sum().sum()
                distancia = np.sqrt(distancia)
            return distancia
        else:
            return None
    def norma(self):
        if True:
            distancia = 0
            for i in range(len(self.red)):
                r0 = (self.red[i][0])**2
                r1 = (self.red[i][1])**2
                distancia = distancia + r0.sum().sum()+r1.sum().sum()
                distancia = np.sqrt(distancia)
            return distancia
        else:
            return None
        
    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    
    def predecir(self,entrada):
        for r in self.red:
            entrada = np.dot(entrada,r[0])
            entrada = entrada + r[1]
            entrada = self.sigmoid(entrada)
        entrada = entrada.round()
        entrada = int(entrada)
        return entrada
    def evaluar(self,muestra):
        target = muestra.target
        u = muestra.columns.tolist()
        v = []
        for i in u:
            if i != 'target':
                v.append(i)
        entrada = muestra.loc[:,v].values.tolist()
        resultado = target.tolist()
        suma = 0
        cuenta = 0
        for i in range(len(entrada)):
            u = self.predecir(entrada[i])
            v = resultado[i]
            if u == v:
                suma += 1
            cuenta += 1
        u = float(suma)/float(cuenta)
        return u  
    def razon(self,other,n_pasos):
        if self.forma == other.forma:
            hijo = binary(self.capas,self.nodos)
            hijo.inicializar()
            red = []
            for i in range(len(self.red)):
                r0 = (-self.red[i][0] + other.red[i][0])/(n_pasos)+self.red[i][0]
                r1 = (-self.red[i][1] + other.red[i][1])/(n_pasos)+self.red[i][1]
                red.append([r0,r1])
            self.red = red
                
            

In [261]:
def crear_traza(individuo,n_pasos,muestra,n_sample,alfa=0.1):
    ''' Devuelve el mejor individuo, el individuo promedio y el ranking
    tanto del mejor individuo como del promedio'''
    prediccion_traza = 0
    mejor_individuo = individuo.clonar()
    mejor_prediccion = mejor_individuo.evaluar(muestra.sample(n_sample))
    for i in range(n_pasos):
        hijo = individuo.mutar(alfa)
        traza = hijo.clonar()
        prediccion = hijo.evaluar(muestra.sample(n_sample))
        if prediccion > mejor_prediccion:
            mejor_prediccion = prediccion
            mejor_individuo = hijo.clonar()
        prediccion_traza += prediccion
        individuo = hijo.clonar()
    prediccion_traza = prediccion_traza/float(n_pasos)
    return mejor_prediccion, mejor_individuo, prediccion_traza, traza

In [262]:
titanic = pd.read_csv('train.csv')
titanic.groupby('Embarked').size()
titanic = pd.read_csv('train.csv')
titanic = titanic[['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
titanic.loc[titanic.Embarked=='C','Embarked']=-1
titanic.loc[titanic.Embarked=='Q','Embarked']=0
titanic.loc[titanic.Embarked=='S','Embarked']=1
titanic['target'] = titanic.Survived
titanic['Pclass_1']=np.where(titanic.Pclass==1,1,0)
titanic['Pclass_2']=np.where(titanic.Pclass==2,1,0)
titanic['Pclass_3']=np.where(titanic.Pclass==3,1,0)
titanic['Sex']=np.where(titanic.Sex=='male',1,0)
titanic = titanic.fillna(titanic.mean())
titanic[['Age']] = titanic[['Age']]/titanic[['Age']].mean()
titanic[['Fare']] = titanic[['Fare']]/titanic[['Fare']].mean()
del titanic['Pclass'],titanic['Survived']


In [263]:
titanic_train = titanic[titanic.target==0].sample(250)\
                .append(titanic[titanic.target==1].sample(250))
titanic_test = titanic.loc[~titanic.index.isin(titanic_train.index),:]

In [264]:
individuo = binary(0,9)
individuo.inicializar()

In [ ]:
mejor_prediccion = 0
n= 100
alpha = np.exp(np.log(10**-5)/n)
for i in range(n):
    mejor_tprediccion = 0
    alfa = alpha ** i
    print(alfa)
    for i in range(100):
        mp,mi,pt,t = crear_traza(individuo,100,titanic_train,100,alfa = alfa)
        if mp > mejor_prediccion:
            mejor_prediccion = mp
            mejor_individuo = mi.clonar()
            
        if mejor_tprediccion < pt:
            mejor_tprediccion = pt
            mejor_traza = t.clonar()
    print('mejor_prediccion',mejor_prediccion,'mejor_tprediccion',mejor_tprediccion)
    print(individuo.norma())
    individuo = mejor_traza.clonar()

1.0
mejor_prediccion 0.81 mejor_tprediccion 0.6572
4.298422128950256
0.8912509381337456
mejor_prediccion 0.86 mejor_tprediccion 0.6943000000000003
27.401709410657244
0.7943282347242816
mejor_prediccion 0.86 mejor_tprediccion 0.7380000000000001
33.86245134815737
0.707945784384138
mejor_prediccion 0.86 mejor_tprediccion 0.7465999999999998
37.242326815164574
0.6309573444801934
mejor_prediccion 0.9 mejor_tprediccion 0.7662000000000002
44.53896874459191
0.5623413251903492
mejor_prediccion 0.9 mejor_tprediccion 0.7659
52.584118437293405
0.5011872336272724
mejor_prediccion 0.9 mejor_tprediccion 0.7653000000000002
51.219468990637154
0.44668359215096326
mejor_prediccion 0.9 mejor_tprediccion 0.7659000000000001
57.05760261329086
0.39810717055349737
mejor_prediccion 0.9 mejor_tprediccion 0.7704000000000002
58.215191371555115
0.3548133892335756
mejor_prediccion 0.9 mejor_tprediccion 0.7671999999999997
60.56832555762383
0.31622776601683805
mejor_prediccion 0.9 mejor_tprediccion 0.7695000000000001
6

mejor_prediccion 0.93 mejor_tprediccion 0.7889999999999998
67.79648425998853
4.466835921509646e-05
mejor_prediccion 0.93 mejor_tprediccion 0.7874
67.79656170738501
3.981071705534986e-05
mejor_prediccion 0.93 mejor_tprediccion 0.7880000000000006
67.7968558008842
3.548133892335766e-05
mejor_prediccion 0.93 mejor_tprediccion 0.7879000000000002
67.79736555032198
3.16227766016839e-05


In [ ]:
best_model = mejor_traza


In [ ]:
best_model.evaluar(titanic_test)

In [ ]:
titanic = pd.read_csv('test.csv')
titanic = titanic[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
titanic.loc[titanic.Embarked=='C','Embarked']=-1
titanic.loc[titanic.Embarked=='Q','Embarked']=0
titanic.loc[titanic.Embarked=='S','Embarked']=1
titanic['Pclass_1']=np.where(titanic.Pclass==1,1,0)
titanic['Pclass_2']=np.where(titanic.Pclass==2,1,0)
titanic['Pclass_3']=np.where(titanic.Pclass==3,1,0)
titanic['Sex']=np.where(titanic.Sex=='male',1,0)
titanic = titanic.fillna(titanic.mean())
titanic[['Age']] = titanic[['Age']]/titanic[['Age']].mean()
titanic[['Fare']] = titanic[['Fare']]/titanic[['Fare']].mean()
del titanic['Pclass']

In [ ]:
scores = []
for index, row in titanic.iterrows():
    scores.append(best_model.predecir(row.values))


In [ ]:
resultado = pd.read_csv('test.csv')
resultado['Survived'] = scores
resultado.Survived = resultado.Survived.round().astype(int)
resultado[['PassengerId','Survived']].to_csv('result.csv',index=False)